In [1]:
# Install required packages
!pip install llama-cpp-python fastapi uvicorn pyngrok

# Download and set up ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.tgz
!tar -xvf ngrok-stable-linux-amd64.tgz
!mv ngrok /usr/local/bin



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.1-cp310-cp310-linux_x86_64.whl size=3485351 sha256=07f05e67f5bcc834af6540d90884781b2038bc8ae6aa8c095b242622bce14856
  Stored in directory: /root/.cache/pip/wheels/f8/b0/a2/f47d952aec7ab061b9e2a345e23a1e1e137beb7891259e3d0c
Successfully built llama-cpp-python
--2024-10-25 10:59:09-- 

In [2]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf?download=true

--2024-10-25 11:05:52--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf?download=true
Resolving huggingface.co (huggingface.co)... 18.172.134.4, 18.172.134.88, 18.172.134.124, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/b0/ca/b0cae82fd4b3a362cab01d17953c45edac67d1c2dfb9fbb9e69c80c32dc2012e/08a5566d61d7cb6b420c3e4387a39e0078e1f2fe5f055f3a03887385304d4bfa?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b-chat.Q4_K_M.gguf%3B+filename%3D%22llama-2-7b-chat.Q4_K_M.gguf%22%3B&Expires=1730113552&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMDExMzU1Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9iMC9jYS9iMGNhZTgyZmQ0YjNhMzYyY2FiMDFkMTc5NTNjNDVlZGFjNjdkMWMyZGZiOWZiYjllNjljODBjMzJkYzIwMTJlLzA4YTU1NjZkNjFkN2NiNmI0MjBjM2U0Mzg3YTM5ZTAwNzhlMWYyZmU1

In [ ]:
import nest_asyncio
nest_asyncio.apply()  # This allows for nested event loops

from fastapi import FastAPI
from llama_cpp import Llama
from pyngrok import ngrok
import uvicorn
from pydantic import BaseModel

class Query(BaseModel):
    prompt: str
    max_tokens: int = 256
    temperature: float = 0.7

app = FastAPI()

# Initialize LLaMA model
llm = Llama(
    model_path="/content/llama-2-7b-chat.Q4_K_M.gguf",
    n_ctx=2048,
    n_threads=4
)

@app.post("/generate")
async def generate(query: Query):
    response = llm(
        query.prompt,
        max_tokens=query.max_tokens,
        temperature=query.temperature
    )
    return {"response": response["choices"][0]["text"]}

# Set up ngrok tunnel
def setup_ngrok():
    # Replace with your ngrok auth token
    ngrok.set_auth_token("2mEZUUvDapXZKa3Jb6GzUpntoFE_31bHVUSi9veyDXh33GwBf")

    # Start ngrok tunnel
    public_url = ngrok.connect(8000)
    print(f"Public URL: {public_url}")
    return public_url

# Run the server in a way that's compatible with Colab
import asyncio
from concurrent.futures import ThreadPoolExecutor

def run_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    server.run()

# Main execution
url = setup_ngrok()
print(f"Your API is now available at: {url}/generate")

# Run the server in a separate thread
with ThreadPoolExecutor() as executor:
    executor.submit(run_server)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /content/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32       

Public URL: NgrokTunnel: "https://29cb-34-44-194-23.ngrok-free.app" -> "http://localhost:8000"
Your API is now available at: NgrokTunnel: "https://29cb-34-44-194-23.ngrok-free.app" -> "http://localhost:8000"/generate


INFO:     Started server process [999]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
llama_perf_context_print:        load time =    3324.19 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     7 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     9 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    8607.66 ms /    16 tokens


INFO:     2406:b400:b4:c038:dbba:66b3:19c8:6cf:0 - "POST /generate HTTP/1.1" 200 OK


Llama.generate: 6 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    3324.19 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   115 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   72823.04 ms /   116 tokens


INFO:     2406:b400:b4:c038:dbba:66b3:19c8:6cf:0 - "POST /generate HTTP/1.1" 200 OK


Llama.generate: 6 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    3324.19 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    35 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   20196.04 ms /    36 tokens


INFO:     2406:b400:b4:c038:dbba:66b3:19c8:6cf:0 - "POST /generate HTTP/1.1" 200 OK
